# Student Loan Risk with Deep Learning

In [25]:
# Imports
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from pathlib import Path
import numpy as np

---

## Prepare the data to be used on a neural network model

### Step 1: Read the `student-loans.csv` file into a Pandas DataFrame. Review the DataFrame, looking for columns that could eventually define your features and target variables.   

In [3]:
# Define the path to the uploaded dataset
file_path = 'student-loans.csv'  # Path for the uploaded file

# Load the CSV file into a Pandas DataFrame
loans_df = pd.read_csv(file_path)

# Display the first few rows of the DataFrame to review the data structure
loans_df.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score,credit_ranking
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0


In [4]:
# Review the data types associated with the columns
loans_df.dtypes

payment_history           float64
location_parameter        float64
stem_degree_score         float64
gpa_ranking               float64
alumni_success            float64
study_major_code          float64
time_to_completion        float64
finance_workshop_score    float64
cohort_ranking            float64
total_loan_score          float64
financial_aid_score       float64
credit_ranking              int64
dtype: object

In [5]:
# Check the credit_ranking value counts
loans_df["credit_ranking"].value_counts()

credit_ranking
1    855
0    744
Name: count, dtype: int64

### Step 2: Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “credit_ranking”. The remaining columns should define the features dataset.

In [6]:
# Define the target set y using the credit_ranking column
y = loans_df['credit_ranking']

# Display a sample of y
y.sample(5)


1453    0
1085    0
1578    1
1251    0
695     1
Name: credit_ranking, dtype: int64

In [7]:
# Define features set X by selecting all columns except credit_ranking
X = loans_df.drop(columns=['credit_ranking'])

# Review the features DataFrame
X.head()


,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4


### Step 3: Split the features and target sets into training and testing datasets.


In [8]:
# Split the preprocessed data into a training and testing dataset with random_state set to 1
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

# Display the shape of the training and testing sets
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((1279, 11), (320, 11), (1279,), (320,))

### Step 4: Use scikit-learn's `StandardScaler` to scale the features data.

In [10]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features training dataset and transform it
X_train_scaled = scaler.fit_transform(X_train)

# Transform the features testing dataset using the same scaler
X_test_scaled = scaler.transform(X_test)

---

## Compile and Evaluate a Model Using a Neural Network

### Step 1: Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras.

> **Hint** You can start with a two-layer deep neural network model that uses the `relu` activation function for both layers.


In [11]:
# Define the number of inputs (features) to the model
input_features = X_train_scaled.shape[1]

# Review the number of features
print(f"Number of input features: {input_features}")

# Create a deep neural network model
model = Sequential([
    Dense(64, activation='relu', input_shape=(input_features,)),  # First hidden layer with 64 neurons
    Dense(32, activation='relu'),  # Second hidden layer with 32 neurons
    Dense(1, activation='sigmoid')  # Output layer for binary classification
])

# Display a summary of the model
model.summary()

Number of input features: 11
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                768       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2,881
Trainable params: 2,881
Non-trainable params: 0
_________________________________________________________________


In [12]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1 = 64

# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer2 = 32

# Define the number of neurons in the output layer (for binary classification)
output_neurons = 1

# Create the deep neural network model
model = Sequential([
    Dense(hidden_nodes_layer1, activation='relu', input_shape=(input_features,)),  # First hidden layer
    Dense(hidden_nodes_layer2, activation='relu'),  # Second hidden layer
    Dense(output_neurons, activation='sigmoid')  # Output layer
])

# Display a summary of the model to review the architecture
model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 64)                768       
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2,881
Trainable params: 2,881
Non-trainable params: 0
_________________________________________________________________


In [13]:
# Create the Sequential model instance
model = Sequential()

# Add the first hidden layer
model.add(Dense(hidden_nodes_layer1, activation='relu', input_shape=(input_features,)))

# Add the second hidden layer
model.add(Dense(hidden_nodes_layer2, activation='relu'))

# Add the output layer to the model, specifying the number of output neurons and activation function
model.add(Dense(output_neurons, activation='sigmoid'))

# Display a summary of the model to review the architecture
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 64)                768       
                                                                 
 dense_7 (Dense)             (None, 32)                2080      
                                                                 
 dense_8 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2,881
Trainable params: 2,881
Non-trainable params: 0
_________________________________________________________________


In [14]:
# Display the Sequential model summary
model.summary()


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 64)                768       
                                                                 
 dense_7 (Dense)             (None, 32)                2080      
                                                                 
 dense_8 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2,881
Trainable params: 2,881
Non-trainable params: 0
_________________________________________________________________


### Step 2: Compile and fit the model using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric.


In [15]:
# Compile the Sequential model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [16]:
# Fit the model using 50 epochs and the training data
history = model.fit(X_train_scaled, y_train, epochs=50, validation_split=0.2, verbose=1)


Epoch 1/50
32/32 [==============================] - 1s 17ms/step - loss: 0.6324 - accuracy: 0.6784 - val_loss: 0.5760 - val_accuracy: 0.7344
Epoch 2/50
32/32 [==============================] - 0s 6ms/step - loss: 0.5500 - accuracy: 0.7214 - val_loss: 0.5385 - val_accuracy: 0.7344
Epoch 3/50
32/32 [==============================] - 0s 6ms/step - loss: 0.5215 - accuracy: 0.7370 - val_loss: 0.5310 - val_accuracy: 0.7461
Epoch 4/50
32/32 [==============================] - 0s 7ms/step - loss: 0.5077 - accuracy: 0.7468 - val_loss: 0.5251 - val_accuracy: 0.7578
Epoch 5/50
32/32 [==============================] - 0s 6ms/step - loss: 0.4945 - accuracy: 0.7576 - val_loss: 0.5195 - val_accuracy: 0.7500
Epoch 6/50
32/32 [==============================] - 0s 7ms/step - loss: 0.4883 - accuracy: 0.7625 - val_loss: 0.5178 - val_accuracy: 0.7461
Epoch 7/50
32/32 [==============================] - 0s 6ms/step - loss: 0.4813 - accuracy: 0.7683 - val_loss: 0.5158 - val_accuracy: 0.7461
Epoch 8/50
32/32 [=

### Step 3: Evaluate the model using the test data to determine the model’s loss and accuracy.


In [17]:
# Evaluate the model loss and accuracy metrics using the test data
loss, accuracy = model.evaluate(X_test_scaled, y_test, verbose=0)

# Display the model loss and accuracy results
print(f"Model Loss: {loss}")
print(f"Model Accuracy: {accuracy}")


Model Loss: 0.5704757571220398
Model Accuracy: 0.7749999761581421


### Step 4: Save and export your model to a keras file, and name the file `student_loans.keras`.


In [18]:
# Set the model's file path
model_file_path = 'student_loans.keras'

# Export your model to a keras file
model.save(model_file_path)

print(f"Model saved to {model_file_path}")


Model saved to student_loans.keras


---
## Predict Loan Repayment Success by Using your Neural Network Model

### Step 1: Reload your saved model.

In [21]:
# Set the model's file path
model_file_path = 'student_loans.keras'

# Load the model into a new object
loaded_model = load_model(model_file_path)

print("Model loaded successfully.")


Model loaded successfully.


### Step 2: Make predictions on the testing data and save the predictions to a DataFrame.

In [26]:
# Make predictions with the test data
predictions = loaded_model.predict(X_test_scaled)

# Round predictions to binary values (0 or 1)
binary_predictions = np.round(predictions).astype(int)

# Display a sample of the predictions
print("Sample Predictions:")
print(binary_predictions[:5])

10/10 [==============================] - 0s 2ms/step
Sample Predictions:
[[0]
 [0]
 [1]
 [1]
 [1]]


In [27]:
# Save the predictions to a DataFrame and round them to binary results
predictions_df = pd.DataFrame({
    'Predicted': np.round(predictions).astype(int).flatten(),
    'Actual': y_test.values
})

# Display a sample of the predictions DataFrame
predictions_df.head()

,Predicted,Actual
0,0,0
1,0,1
2,1,1
3,1,1
4,1,1


### Step 4: Display a classification report with the y test data and predictions

In [28]:
# Print the classification report with the y_test data and predictions
print("Classification Report:")
print(classification_report(y_test, predictions_df['Predicted']))

Classification Report:
              precision    recall  f1-score   support

           0       0.79      0.72      0.76       154
           1       0.76      0.83      0.79       166

    accuracy                           0.78       320
   macro avg       0.78      0.77      0.77       320
weighted avg       0.78      0.78      0.77       320



---
## Discuss creating a recommendation system for student loans

Briefly answer the following questions in the space provided:

1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.

2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.

3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.

**1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.**
I think we need data GPA or Academic Performance because it will help us with the recommendation. which loan will give out to student with a good GPA or Academic performance because we can collect the loan after they finisning their school. Beside that we should have a lot data of credit history too it will help in the future too


**2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.**
I think the methid Justification for Content-Based Filtering will suit for loan and interest in this data because  Given the collected data—academic performance, financial data, loan and credit history, demographics, and career aspirations. The system can match students to loans with terms that best fit their unique financial profiles and repayment capabilities


**3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.**
I think the real-world challenges that take into cinsideration while building a recommendation system for students loans is privacy and fairness for people who recieving the loan and rejected the loan. 
